Attempt at production ready version of life_insurance_pricing_practice.ipynb
Logic will be used in .py later and integrated with interface for data entry

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import newton

# Loads data from provided VBT Excel file and creates a map of Age/Policy duration -> Mortality
def load_mortality_data(file_name, sheet_name):
  df_raw_vbt = pd.read_excel(file_name, sheet_name=sheet_name, header=None)

  header_row_idx = df_raw_vbt[df_raw_vbt[0] == r"Row\Column"].index[0]
  df_vbt = pd.read_excel(file_name, sheet_name=sheet_name, header = header_row_idx)

  df_vbt = df_vbt.rename(columns={r"Row\Column": "Issue_Age"})
  df_vbt = df_vbt[pd.to_numeric(df_vbt["Issue_Age"], errors='coerce').notnull()]

  df_vbt["Issue_Age"] = df_vbt["Issue_Age"].astype(int)
  df_vbt = df_vbt.dropna()

  df_long = df_vbt.melt(
    id_vars=["Issue_Age"],
    var_name="Duration",
    value_name="Mortality_Rate"
  )

  df_long["Duration"] = pd.to_numeric(df_long["Duration"], errors='coerce')
  # Convert Mortality_Rate to numeric type
  df_long["Mortality_Rate"] = pd.to_numeric(df_long["Mortality_Rate"], errors='coerce')

  mortality_map = df_long.set_index(["Issue_Age", "Duration"])["Mortality_Rate"].to_dict()

  print(f"Loading data from {file_name}...")
  return mortality_map

mortality_lookup = load_mortality_data("VBT_2015.xlsx", "Sheet1")

product_A_assumptions = {
    "projection_years": 20,
    "interest_rate": 0.05,
    "lapse_vector": [0.1, 0.08] + [0.05]*18,
    "expenses": {
        "commission": 150,
        "maintenance": 50
    },
    "inflation_rate": 0.02
}

product_B_assumptions = {
    "projection_years": 20,
    "interest_rate": 0.03,
    "lapse_vector": [0.2, 0.15] + [0.10]*18,
    "expenses": {
        "commission": 100,
        "maintenance": 40
    },
    "inflation_rate": 0.02,
}



class LifePolicy:
    def __init__(self, age, l0, premium, claims_amount, mortality_map, assumptions):
        # Key assumptions passed
        self.age = age
        self.premium = premium
        self.claims = claims_amount
        self.mortality_map = mortality_map
        self.l0 = l0

        # Base assumptions from config dict
        self.projection_years = assumptions["projection_years"]
        self.interest_rate = assumptions["interest_rate"]
        self.lapse_rates = assumptions["lapse_vector"]
        self.expenses_comm = assumptions["expenses"]["commission"]
        self.expenses_maint = assumptions["expenses"]["maintenance"]

        # Result placeholders
        self.df_cashflow = None
        self.npv = 0.0
        self.profit_margin = 0.0


    def _get_mortality_vector(self):
        return [self.mortality_map.get((self.age, d), 0.5) for d in range(1, self.projection_years + 1)]

    def project_cashflows(self, premium):
        # ... logic to build df_cashflow ...
        df_cashflow = pd.DataFrame(range(1, self.projection_years + 1))
        df_cashflow.columns = ["Year"]

        q_x = self._get_mortality_vector()

        df_cashflow["Mortality Rate"] = q_x
        df_cashflow["Lapse Rate"] = self.lapse_rates
        df_cashflow["Net Retention"] = (1 - df_cashflow["Mortality Rate"]) * (1 - df_cashflow["Lapse Rate"])
        df_cashflow["End Lives"] = self.l0 * df_cashflow["Net Retention"].cumprod()

        return df_cashflow

    def get_npv(self, premium):
        # ... returns scalar NPV ...
        return scalar_npv

test = LifePolicy(35, 1000, 500, 500000, mortality_map=mortality_lookup, assumptions=product_A_assumptions)

print(test.project_cashflows(500))

Loading data from VBT_2015.xlsx...
    Year  Mortality Rate  Lapse Rate  Net Retention   End Lives
0      1         0.00015        0.10       0.899865  899.865000
1      2         0.00018        0.08       0.919834  827.726782
2      3         0.00029        0.05       0.949724  786.112405
3      4         0.00034        0.05       0.949677  746.552870
4      5         0.00040        0.05       0.949620  708.941536
5      6         0.00045        0.05       0.949573  673.191387
6      7         0.00051        0.05       0.949515  639.205656
7      8         0.00062        0.05       0.949411  606.868882
8      9         0.00070        0.05       0.949335  576.121870
9     10         0.00076        0.05       0.949278  546.899816
10    11         0.00083        0.05       0.949211  519.123595
11    12         0.00092        0.05       0.949126  492.713701
12    13         0.00104        0.05       0.949012  467.591215
13    14         0.00117        0.05       0.948888  443.691927
14   